In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import shap

In [2]:
data = pd.read_csv('./data/data_all.csv')
data.head()

,粗醇(t),精醇(t),二甲醚(t),醋酸(t),一厂乙二醇(t),二厂乙二醇(t),液体产品(t),乙腈(t),发电量(万度),一期有效气(万Nm3),...,二期原煤(t),下网电量(万kwh),一期用电量(万度),二期用电量(万度),预脱盐水(t),软化水(t),市政污水处理量(m3/d),新城提升泵供水(m3/d),一期酸脱年耗醇,二期酸脱年耗醇
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,23.0,6.3,17.2,4342.0,44718.0,1336.0,7700.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,25.0,6.4,18.1,2426.0,52730.0,1726.0,8801.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,24.0,6.1,17.7,1248.0,41558.0,2104.0,7970.0,0.0,0.0


In [3]:
new_columns = ['cuchun', 'jingchun', 'erjiami', 'cusuan', 'yichangyierchun', 'erchangyierchun', 'yetichanpin', 'yijing', 'fadianliang', 'yiqiyouxiaoqi', 'erqiyouxiaoqi', 'yierchunqingzufen', 'yierchunzhongzufen', 'ranmei', 'yiqiyuanmei', 'erqiyuanmei', 'xiawangdianliang', 'yiqiyongdianliang', 'erqiyongdianliang', 'yutuoyanshui', 'ranhuashui', 'shizhengwushuichuliliang', 'xinchengtishengbenggongshui', 'yiqisuantuonianhaochun', 'erqisuantuonianhaochun']

data.columns = new_columns
data = data.fillna(0)
data.head()

,cuchun,jingchun,erjiami,cusuan,yichangyierchun,erchangyierchun,yetichanpin,yijing,fadianliang,yiqiyouxiaoqi,...,erqiyuanmei,xiawangdianliang,yiqiyongdianliang,erqiyongdianliang,yutuoyanshui,ranhuashui,shizhengwushuichuliliang,xinchengtishengbenggongshui,yiqisuantuonianhaochun,erqisuantuonianhaochun
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,23.0,6.3,17.2,4342.0,44718.0,1336.0,7700.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,25.0,6.4,18.1,2426.0,52730.0,1726.0,8801.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,...,0.0,24.0,6.1,17.7,1248.0,41558.0,2104.0,7970.0,0.0,0.0


In [11]:
from EDAandPCA import get_pca_features_from_df
from sklearn.model_selection import train_test_split
np.random.seed(45)

X = data.iloc[:, 0:13]
X = get_pca_features_from_df(X)
Y = data.iloc[:, 13:25]


X.shape, Y.shape

((365, 4), (365, 12))

In [12]:
X_train, X_holdout, Y_train, Y_holdout = train_test_split(X, Y, test_size=0.1)

In [13]:
import sklearn.linear_model as lm

linear_model = lm.LinearRegression()

# Fit your linear model
linear_model.fit(X_train, Y_train)

# Predict housing prices on the test set
Y_pred = linear_model.predict(X_holdout)

In [14]:
def rmse(actual_y, predicted_y):
    return np.sqrt(np.mean((predicted_y - actual_y)**2))

train_error = rmse(Y_train, linear_model.predict(X_train))
holdout_error = rmse(Y_holdout, Y_pred)

print("Training RMSE:", train_error)
print("Holdout RMSE:", holdout_error)


Training RMSE: 2135.6236250891857
Holdout RMSE: 2119.2990014723973


## SHAP display

In [15]:

background = shap.utils.sample(X, 100)
explainer = shap.Explainer(linear_model.predict, background)
shap_values = explainer(X)

In [16]:
shap_values

.values =
array([[[-9.73105991e+02, -1.62607265e+03,  4.96230771e+02, ...,
         -8.70749436e+02, -2.56527309e+02, -4.11859454e+02],
        [-6.12624732e+02,  6.17008161e+02, -2.13529402e+03, ...,
          1.94225936e+02, -7.54091520e+01, -1.21898494e+02],
        [-9.61778952e+01, -1.59168757e+02, -2.06232216e+01, ...,
         -4.13847880e+02,  1.64060044e+01,  2.79910799e+01],
        [-5.32825683e+01,  2.42481532e+01,  6.94291398e+01, ...,
          3.01204610e+01, -6.62713365e+00, -2.10075168e+01]],

       [[-9.73105991e+02, -1.62607265e+03,  4.96230771e+02, ...,
         -8.70749436e+02, -2.56527309e+02, -4.11859454e+02],
        [-6.12624732e+02,  6.17008161e+02, -2.13529402e+03, ...,
          1.94225936e+02, -7.54091520e+01, -1.21898494e+02],
        [-9.61778952e+01, -1.59168757e+02, -2.06232216e+01, ...,
         -4.13847880e+02,  1.64060044e+01,  2.79910799e+01],
        [-5.32825683e+01,  2.42481532e+01,  6.94291398e+01, ...,
          3.01204610e+01, -6.62713365e+00

### Create the summary plot for a specific output/label/target.